In [1]:
import pandas as pd
import numpy as np
import pyodbc
import os
t_start0 = pd.Timestamp.now()
print(t_start0)

2023-02-17 15:22:38.498813


In [2]:
def rmmk(dir): # remove exiting folder and remake folder
    import os
    import shutil

    if os.path.exists(dir):
        shutil.rmtree(dir)
    os.makedirs(dir)

In [3]:
def printAllContents(ctx, relativeUrl):

    try:
        
        libraryRoot = ctx.web.get_folder_by_server_relative_url(relativeUrl)
        ctx.load(libraryRoot)
        ctx.execute_query()

        folders = libraryRoot.folders
        ctx.load(folders)
        ctx.execute_query()

        for myfolder in folders:
            #print("Folder name: {0}".format(myfolder.properties["Name"]))
            print("Folder name: {0}".format(myfolder.properties["ServerRelativeUrl"]))
            printAllContents(ctx, relativeUrl + '/' + myfolder.properties["Name"])
            
        files = libraryRoot.files
        ctx.load(files)
        ctx.execute_query()

        for myfile in files:
            print("File name: {0}".format(myfile.properties["ServerRelativeUrl"]))
            pathList = myfile.properties["ServerRelativeUrl"].split('/')
            fileDest = outputDir + "/"+ pathList[-1]
            downloadFile(ctx, fileDest, myfile.properties["ServerRelativeUrl"])
            
    except:
        
        print('Problem printing out list of folders')   
        sys.exit(1)  

In [4]:
def save_txt(src,dst):
    with open(os.path.join(dst), 'a', encoding ='utf8') as txt_file:
            txt_file.write(src+'\n')

In [5]:
# https://gist.github.com/hyunjong-lee/e0f97aa0e4c353c65c31b596765ee641
import os
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.files.file_system_object_type import FileSystemObjectType
from office365.sharepoint.files.file import File
from tqdm.notebook import tqdm
import keyring

sharepoint_user = keyring.get_password("sharepointonline_username", "username")
sharepoint_pass = keyring.get_password("sharepointonline_pass","password")

baseurl = 'https://irpcplc.sharepoint.com'
basesite = '/sites/FLEQBOM' # every share point has a home.
docurl = '/Shared%20Documents/User_keydata'
siteurl = baseurl + basesite 
# target_url = basesite+docurl

ctx_auth = AuthenticationContext(siteurl)
ctx_auth.acquire_token_for_user(sharepoint_user, sharepoint_pass)
ctx = ClientContext(siteurl, ctx_auth) # make sure you auth to the siteurl.

doc_lib = ctx.web.lists.get_by_title("Documents")
items = doc_lib.items.select(["FileSystemObjectType"]).expand(["File", "Folder"]).get().execute_query()
folders = []
files = []
dst_dir = './sharepointindex'
rmmk(dst_dir)
for item in tqdm(items):  # type: ListItem
    if item.file_system_object_type == FileSystemObjectType.Folder:
        print("Folder url: {0}".format(item.folder.serverRelativeUrl))
        src_folder = item.folder.serverRelativeUrl
        save_txt(src_folder,os.path.join(dst_dir,'sharepointfolders.txt'))
        folders.append(item.folder.serverRelativeUrl)
    else:
        print("File url: {0}".format(item.file.serverRelativeUrl))
        src_file = item.file.serverRelativeUrl
        save_txt(src_file,os.path.join(dst_dir,'sharepointfiles.txt'))
        files.append(item.file.serverRelativeUrl)

  0%|          | 0/100 [00:00<?, ?it/s]

Folder url: /sites/FLEQBOM/Shared Documents/ที่มาที่ไป
Folder url: /sites/FLEQBOM/Shared Documents/SAP_connection
Folder url: /sites/FLEQBOM/Shared Documents/Exported
Folder url: /sites/FLEQBOM/Shared Documents/Form
Folder url: /sites/FLEQBOM/Shared Documents/old_4users
Folder url: /sites/FLEQBOM/Shared Documents/Plan
Folder url: /sites/FLEQBOM/Shared Documents/4users
Folder url: /sites/FLEQBOM/Shared Documents/User_keydata
Folder url: /sites/FLEQBOM/Shared Documents/User_keydata/-
Folder url: /sites/FLEQBOM/Shared Documents/User_keydata/11E
Folder url: /sites/FLEQBOM/Shared Documents/User_keydata/11I
Folder url: /sites/FLEQBOM/Shared Documents/User_keydata/11M
Folder url: /sites/FLEQBOM/Shared Documents/User_keydata/12E
Folder url: /sites/FLEQBOM/Shared Documents/User_keydata/12I
Folder url: /sites/FLEQBOM/Shared Documents/User_keydata/13E
Folder url: /sites/FLEQBOM/Shared Documents/User_keydata/13I
Folder url: /sites/FLEQBOM/Shared Documents/User_keydata/13M
Folder url: /sites/FLEQBO

In [6]:
# https://stackoverflow.com/questions/53671547/python-download-files-from-sharepoint-site
# response = File.open_binary(ctx, f"/{filtered[1].split('/',3)[-1]}")
# with open(f"./user_keydata/{filtered[1].split('/')[-1]}", "wb") as local_file:
#     local_file.write(response.content)

In [7]:
# https://python.plainenglish.io/list-download-files-from-sharepoint-using-python-44cfc0793397
# file_url = '/sites/FLEQBOM/Shared Documents/old_4users/11I.xlsx'
# # response = File.open_binary(ctx, f"/{filtered[1].split('/',3)[-1]}")
# with open(f"./user_keydata/{filtered[1].split('/')[-1]}", "wb") as local_file:
#     file = ctx.web.get_file_by_server_relative_url(file_url)
#     file.download(local_file)
#     ctx.execute_query()

In [8]:
# https://stackoverflow.com/questions/53671547/python-download-files-from-sharepoint-site
# rmmk('./user_keydata')
# filtered = filter(lambda files: 'User_keydata' in files, files)
# filtered = list(filtered)
# for i,f in enumerate(filtered):
#     print(i,f)
#     if f.split('/')[-1][0] <= 'S':
#         response = File.open_binary(ctx, f"/{f.split('/',3)[-1]}")
#         with open(f"./user_keydata/{f.split('/')[-1]}", "wb") as local_file:
#             local_file.write(response.content)

In [9]:
# https://stackoverflow.com/questions/53671547/python-download-files-from-sharepoint-site
rmmk('./user_keydata')
filtered = filter(lambda files: 'User_keydata' in files, files)
filtered = list(filtered)
for i,f in enumerate(filtered):
    print(i,f)
    if f.split('/')[-1][0] <= 'S':
        with open(f"./user_keydata/{f.split('/')[-1]}", "wb") as local_file:
            file = ctx.web.get_file_by_server_relative_url(f)
            file.download(local_file)
            ctx.execute_query()

0 /sites/FLEQBOM/Shared Documents/User_keydata/-/-.xlsx
1 /sites/FLEQBOM/Shared Documents/User_keydata/11E/11E.xlsx
2 /sites/FLEQBOM/Shared Documents/User_keydata/11I/11I.xlsx
3 /sites/FLEQBOM/Shared Documents/User_keydata/11M/11M.xlsx
4 /sites/FLEQBOM/Shared Documents/User_keydata/12E/12E.xlsx
5 /sites/FLEQBOM/Shared Documents/User_keydata/12I/12I.xlsx
6 /sites/FLEQBOM/Shared Documents/User_keydata/13E/13E.xlsx
7 /sites/FLEQBOM/Shared Documents/User_keydata/13I/13I.xlsx
8 /sites/FLEQBOM/Shared Documents/User_keydata/13M/13M.xlsx
9 /sites/FLEQBOM/Shared Documents/User_keydata/14E/14E.xlsx
10 /sites/FLEQBOM/Shared Documents/User_keydata/14M/14M.xlsx
11 /sites/FLEQBOM/Shared Documents/User_keydata/15C/15C.xlsx
12 /sites/FLEQBOM/Shared Documents/User_keydata/21E/21E.xlsx
13 /sites/FLEQBOM/Shared Documents/User_keydata/21I/21I.xlsx
14 /sites/FLEQBOM/Shared Documents/User_keydata/21M/21M.xlsx
15 /sites/FLEQBOM/Shared Documents/User_keydata/23M/23M.xlsx
16 /sites/FLEQBOM/Shared Documents/Use

# Make dataframe to upload to sharepoint another folder

In [10]:
xlsx = os.listdir('./user_keydata')

In [11]:
df = pd.DataFrame()
for f in tqdm(xlsx):
    df1 = pd.read_excel(f'./user_keydata/{f}',sheet_name=f'{f[:-5]}')
    df1 = df1[df1.columns[~df1.columns.str.startswith('Unnamed')]]
    # df1['ExpectedPG'] = df1.apply(lambda v: v.plannerGroup_eq if v.New_PG.isnull() else v.New_PG)
    df1 = df1.assign(ExpectedPG=np.where(df1.New_PG.isnull(), df1.plannerGroup_eq, df1.New_PG))
    df1['file'] = f
    df = df.append(df1)

  0%|          | 0/50 [00:00<?, ?it/s]

d:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [12]:
rmmk('./4dashboard')
df.to_excel('./4dashboard/FLEQ.xlsx', index=False)

# Upload to sharepoint

In [13]:
# Upload appended files to sharepoint another folder
baseurl = 'https://irpcplc.sharepoint.com'
basesite = '/sites/FLEQBOM' # every share point has a home.
docurl = '/Shared%20Documents'
siteurl = baseurl + basesite 
target_url = basesite+docurl
ctx_auth = AuthenticationContext(siteurl)
ctx_auth.acquire_token_for_user(sharepoint_user, sharepoint_pass)
ctx = ClientContext(siteurl, ctx_auth) # make sure you auth to the siteurl.

# target_url = basesite+docurl
target_folder = ctx.web.get_folder_by_server_relative_url(target_url)

size_chunk = 250*1024**2

local_path = ''

# file_size = os.path.getsize(local_path)


def print_upload_progress(offset):
    file_size = os.path.getsize(local_path)
    print("Uploaded '{}' bytes from '{}'...[{}%]".format(offset, file_size, round(offset / file_size * 100, 2)))
def upload_largefile2sharepoint(file2upload=local_path):
    t_start=pd.Timestamp.now()
    uploaded_file = target_folder.files.create_upload_session(file2upload, size_chunk, print_upload_progress).execute_query()
    print('File {0} has been uploaded successfully'.format(uploaded_file.serverRelativeUrl))
    print(f'Time2Upload {file2upload}: {pd.Timestamp.now()-t_start}')

In [14]:
files = os.listdir('./4dashboard/')
for f in tqdm(files):
    # target_folder = ctx.web.folders.add(str(f'{docurl[1:]}/{f[:-5]}'))
    target_folder = ctx.web.folders.add(f'{docurl[1:]}/4Dashboard')
    ctx.execute_query()
    local_path = f'./4dashboard/{f}'
    upload_largefile2sharepoint(local_path)

  0%|          | 0/1 [00:00<?, ?it/s]

File /sites/FLEQBOM/Shared Documents/4Dashboard/FLEQ.xlsx has been uploaded successfully
Time2Upload ./4dashboard/FLEQ.xlsx: 0 days 00:00:02.867529


In [15]:
print(f'All of process run time = {pd.Timestamp.now()-t_start0}')

All of process run time = 0 days 00:04:34.504897
